# Building the main pathfinding function
---

In [1]:
import numpy as np
import pandas as pd

In [2]:
import heapq as hq

In [3]:
# table of stop data (prefiltered to those in service between regular weekdays from 5pm - 6pm)
stops = pd.read_pickle('data/model/model_stops.pickle')

In [4]:
# walking speed factor
wsf = pd.read_pickle('data/model/f.pickle')

In [5]:
# preprocessed reference tables
E = pd.read_pickle('data/model/list_of_neighbors.pickle')
W = pd.read_pickle('data/model/weights.pickle')

In [6]:
stops.head(3)

,stop_id,stop_code,stop_name,stop_lat,stop_lon
1,263,929,Davenport Rd at Bedford Rd,43.674448,-79.399659
2,264,940,Davenport Rd at Dupont St,43.675511,-79.401938
3,265,1871,Davisville Ave at Cleveland St,43.702088,-79.378112


In [7]:
stop_id_array = stops.stop_id.unique()

In [8]:
# search to stops df for the closest stop id given a set of geocoordinates
def find_closest_stop_id(input_lat, input_lon):
    stop_distance = stops.loc[:, ['stop_id', 'stop_lat', 'stop_lon']]
    stop_distance['distance'] = ( abs(input_lat - stop_distance['stop_lat'])**2 + abs(input_lon - stop_distance['stop_lon'])**2 )**(1/2)
    closest_stop_id = stop_distance.sort_values(by = 'distance').stop_id.iloc[0]
    return closest_stop_id

In [9]:
# currently set to home address
s_location = (43.760442381532236, -79.33181073515874)

In [10]:
# currently set to North York General Hospital near Leslie & Sheppard
t_location = (43.769136021058905, -79.3627326936882)

In [11]:
# s => source, t => target
s_stop_id = find_closest_stop_id(s_location[0], s_location[1])
t_stop_id = find_closest_stop_id(t_location[0], t_location[1])

s_stop_id, t_stop_id

(917, 2592)

In [12]:
# confirm the selection of s & t stops
stops[(stops.stop_id == s_stop_id) | (stops.stop_id == t_stop_id)]

,stop_id,stop_code,stop_name,stop_lat,stop_lon
436,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751
1835,2592,13688,North York General Hospital - Main Entrance,43.769710,-79.363485


In [13]:
# create an empty graph df for pathfinding model to build upon
graph = stops.copy()
graph = graph.set_index('stop_id')
graph = graph.drop(columns = ['stop_code', 'stop_name'])
graph['duration'] = np.nan
graph['parent'] = np.nan
graph['transit'] = False

In [14]:
graph.head(3)

,stop_lat,stop_lon,duration,parent,transit
stop_id,,,,,
263,43.674448,-79.399659,NaN,NaN,False
264,43.675511,-79.401938,NaN,NaN,False
265,43.702088,-79.378112,NaN,NaN,False


In [15]:
# x = source, y = walking speed factor
def initialize_graph(df, x, y):
    df2 = df.copy()
    lat = df.loc[x, 'stop_lat']
    lon = df.loc[x, 'stop_lon']
    distance = ((abs(df.stop_lat - lat)**2 + abs(df.stop_lon - lon)**2)**(1/2))
    df.duration = distance / y
    df2.duration = np.Inf
    df2.loc[x, 'duration'] = 0
    df2.parent = 0
    return df2[['duration', 'parent', 'transit']], df.duration

In [29]:
# x = source, y = walking speed factor
def initialize_graph2(df, x, y):
    df.duration = np.Inf
    df.loc[x, 'duration'] = 0
    df.parent = 0
    return df[['duration', 'parent', 'transit']]

In [16]:
def initialize_priority_queue(graph):
    duration = graph.duration.values
    index = graph.index.values
    counter = len(graph)
    queue = list(list(l) for l in zip(duration, range(0, counter), index))
    stop_lookup = dict(zip(index, queue))
    hq.heapify(queue)
    return queue, stop_lookup, counter

In [17]:
def update_priority_queue(queue, stop_lookup, counter, stop, duration):
    if stop in stop_lookup:
        mark_stop_as_invalid(stop_lookup, stop)
    counter += 1
    stop_entry = [duration, counter, stop]
    stop_lookup[stop] = stop_entry
    hq.heappush(queue, stop_entry)
    return queue, stop_lookup, counter

In [18]:
def mark_stop_as_invalid(stop_lookup, stop):
    stop_entry = stop_lookup.pop(stop)
    stop_entry[-1] = 'invalid_entry'
    return stop_lookup

In [19]:
def extract_min_from_priority_queue(queue, stop_lookup):
    while queue:
        duration, count, stop = hq.heappop(queue)
        if stop != 'invalid_entry':
            del stop_lookup[stop]
            return queue, stop_lookup, stop
    else:
        return queue, stop_lookup, 0

# Next Steps to Finish Main Function
- final preprocessing to produce E & W reference tables
- create a function to calculate the H reference table
- create the relax function

In [20]:
def path_finder(G, s, t, f):
    
    G, H = initialize_graph(G, s, f)
    
    Q, Qd, c = initialize_priority_queue(G)
    
    while Q:
        
        Q, Qd, u = extract_min_from_priority_queue(Q, Qd)
        
        if u == t:
            print('found t')
            return G
        
        for v in E.loc[u]:
            
            w, m = W.loc[(u, v)]
            
            h = H.loc[u]
            
            G, Q, Qd, c = relax(u, v, w, m, h, G, Q, Qd, c)
    
    return G

In [27]:
def path_finder2(G, s, t, f):
    
    G = initialize_graph2(G, s, f)
    
    Q, Qd, c = initialize_priority_queue(G)
    
    while Q:
        
        Q, Qd, u = extract_min_from_priority_queue(Q, Qd)
        
        if u == t:
            print('found t')
            return G
        
        for v in E.loc[u]:
            
            w, m = W.loc[(u, v)]
                        
            G, Q, Qd, c = relax(u, v, w, m, 0, G, Q, Qd, c)
    
    return G

In [21]:
def relax(stop, neighbor, weight, transit, heuristic, graph, queue, stop_lookup, counter):
    alternate = graph.loc[stop, 'duration'] + weight + heuristic
    if alternate < graph.loc[neighbor, 'duration']:
        graph.loc[neighbor, 'duration'] = alternate
        graph.loc[neighbor, 'parent'] = stop
        graph.loc[neighbor, 'transit'] = transit
        queue, stop_lookup, counter = update_priority_queue(queue, stop_lookup, counter, neighbor, alternate)
    return graph, queue, stop_lookup, counter

In [ ]:
%%timeit
g = path_finder2(graph, s_stop_id, 0, wsf)

found t
found t
found t
found t
found t
found t


In [ ]:
# Aamir's house - 73 Gladstone Ave, Toronto, ON
t_stop_id2 = find_closest_stop_id(43.644801355063294, -79.4276479483234)

In [ ]:
t_stop_id2

In [ ]:
%%timeit
g = path_finder(graph, s_stop_id, t_stop_id2, wsf)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f path_finder path_finder(graph, s_stop_id, 0, wsf)

In [ ]:
g = path_finder(graph, s_stop_id, t_stop_id2, wsf)

In [ ]:
g.info()

In [ ]:
def shortest_path(G, s, t):
    S = []
    u = t
    if G.loc[u, 'parent'] != 0:
        while u > 0:
            S.insert(0, u)
            u = G.loc[u, 'parent']
    return S

In [ ]:
s = shortest_path(g, s_stop_id, t_stop_id2)

In [ ]:
s

In [ ]:
path = pd.DataFrame(columns=stops.columns)

In [ ]:
path

In [ ]:
for i in s:
    path = pd.concat([path, stops.loc[stops.stop_id == i]])

In [ ]:
path = path.reset_index(drop = True)

In [ ]:
path

In [ ]:
import folium
from branca.element import Figure

In [ ]:
stops_map = folium.Map(location = [path.stop_lat.mean(), path.stop_lon.mean()], zoom_start = 12, width = 800, height = 600)

In [ ]:
for i in range(0, len(path)):
#     marker_color = ''
#     if (stops_sample.wheelchair_boarding[i] == 1):
#         marker_color = 'green'
#     else:
#         marker_color = 'red'
    folium.Marker(
                    location = [path.stop_lat[i], path.stop_lon[i]],
                    popup = path.stop_name[i],
                    tooltip = path.stop_id[i],
                    icon = folium.Icon(color = 'red', prefix = 'fa', icon = 'bus')
                 ).add_to(stops_map)

In [ ]:
fig = Figure(width = 800, height = 600)
fig.add_child(stops_map)
fig